In [95]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [96]:
all_property = []

for page in range(21,29):
    print('Page: ', page)
    link = 'https://www.apartments.com/apartments/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
#     print(soup)
    results = soup.find_all('article', class_='placard placard-option has-header js-diamond')
#     print(results, ' testing')
    
    # Loop through returned results and get the hyperlink of the properties of the page
    for result in results:
#         print(result)
        find_tag = result.find('a')['href']
        print('find tag: ', find_tag)
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
#         print('new_results: ', new_results)
        
        full_address = result.find('div', class_ = "property-address").text
#         print('full_address: ', full_address)
        street = new_results.find('div', class_ = "propertyAddressContainer").select('span')[0].text
        city = new_results.find('div', class_ = "propertyAddressContainer").select('span')[1].text
        state = new_results.find('div', class_ = "propertyAddressContainer").select('span')[3].text
#         print('state: ', state)
        zip_code = new_results.find('div', class_ = "propertyAddressContainer").select('span')[4].text
#         print('zip_code: ', zip_code)
        property_name = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
#         print('property_name: ', property_name)
        bedroom_container = new_results.find('section', class_="availabilitySection")
#         print('bedroom_container: ', bedroom_container)
        all_the_rents = bedroom_container.findAll('div', class_="priceBedRangeInfo")
#         print('all_the_rents: ', all_the_rents)
        
        rent_obj = {}
        for rent_block in all_the_rents:
            price = rent_block.find('span', class_ = "rentLabel").text.strip()
            if "–" in price:
                price = price.split()[0]
            if "Call for Rent" in price or "/" in price:
                continue
            float_price = float(price.replace("$", "").replace(",", ""))
#             print('rent_block: ', rent_block)
#             print('price: ', price)
            testing = rent_block.find('h4', class_ = "detailsLabel").find('span', class_ = "leaseDepositLabel").text
            if "Not Available" in testing:
                continue
            bedroom_type = rent_block.find('h4', class_ = "detailsLabel").text.strip().split(",")[0]
            print(bedroom_type, " ", float_price)
            if bedroom_type in rent_obj and rent_obj[bedroom_type] < float_price:
                continue
            else:
                rent_obj[bedroom_type] = float_price
        rent_obj["Property_Name_Rent"] = property_name
        rent_obj["City_Rent"] = city
        rent_obj["Type"] = "Apartment"
        rent_obj["State_Rent"] = state
        rent_obj["Zip_code"] = zip_code
        rent_obj["Full_Address"] = full_address
        rent_obj["Rent_link"] = find_tag
        
        all_property.append(rent_obj)
#         print(all_property)
        time.sleep(3)
        
# pprint(all_property);

Page:  21
find tag:  https://www.apartments.com/solana-at-grand-escondido-ca/gp0gp1y/
1 bed   2075.0
2 beds   2365.0
2 beds   2445.0
2 beds   2445.0
2 beds   2375.0
2 beds   2450.0
1 bed   2075.0
2 beds   2365.0
2 beds   2445.0
2 beds   2445.0
2 beds   2375.0
2 beds   2450.0
find tag:  https://www.apartments.com/the-summit-at-chino-hills-chino-hills-ca/m9e7x7j/
1 bed   2250.0
2 beds   2680.0
1 bed   2250.0
2 beds   2680.0
find tag:  https://www.apartments.com/river-terrace-santa-clara-ca/dcr4v1l/
1 bed   2585.0
1 bed   2566.0
1 bed   2470.0
2 beds   2849.0
2 beds   2987.0
2 beds   2989.0
1 bed   2585.0
1 bed   2566.0
1 bed   2470.0
2 beds   2849.0
2 beds   2987.0
2 beds   2989.0
find tag:  https://www.apartments.com/park-bonita-bonita-ca/yx7nyqv/
2 beds   2666.0
2 beds   2666.0
find tag:  https://www.apartments.com/hollypark-knolls-inglewood-ca/3k3yh1f/
Studio   1655.0
1 bed   1609.0
1 bed   1747.0
1 bed   1719.0
2 beds   2373.0
2 beds   2057.0
2 beds   2115.0
Studio   1655.0
1 bed   1

find tag:  https://www.apartments.com/saybrook-pointe-san-jose-ca/mdj42yl/
1 bed   2450.0
2 beds   2980.0
3 beds   3850.0
1 bed   2450.0
2 beds   2980.0
3 beds   3850.0
find tag:  https://www.apartments.com/preserve-at-marin-corte-madera-ca/q7gqk7x/
2 beds   4357.0
3 beds   4230.0
3 beds   4775.0
2 beds   4357.0
3 beds   4230.0
3 beds   4775.0
find tag:  https://www.apartments.com/pacific-bay-vistas-apartments-san-bruno-ca/m3d3jlk/
1 bed   2545.0
1 bed   2855.0
2 beds   3485.0
2 beds   3405.0
1 bed   2545.0
1 bed   2855.0
2 beds   3485.0
2 beds   3405.0
find tag:  https://www.apartments.com/sonterra-at-foothill-ranch-foothill-ranch-ca/4mjkn7l/
1 bed   2547.0
1 bed   2491.0
2 beds   2857.0
2 beds   3034.0
2 beds   3202.0
1 bed   2547.0
1 bed   2491.0
2 beds   2857.0
2 beds   3034.0
2 beds   3202.0
find tag:  https://www.apartments.com/the-oaks-santa-clarita-ca/j3qt204/
1 bed   2360.0
1 bed   2235.0
1 bed   2330.0
1 bed   2057.0
1 bed   2145.0
2 beds   3220.0
2 beds   2860.0
2 beds   327

2 beds   2140.0
2 beds   2365.0
2 beds   2140.0
2 beds   2365.0
find tag:  https://www.apartments.com/atlantic-times-square-monterey-park-ca/2s5f8c8/
3 beds   3500.0
3 beds   3500.0
find tag:  https://www.apartments.com/the-aspens-riverside-riverside-ca/nq1v8gn/
1 bed   1925.0
1 bed   1925.0
1 bed   1925.0
1 bed   1925.0
find tag:  https://www.apartments.com/glen-forest-apartments-anaheim-ca/p7bd98r/
1 bed   1605.0
1 bed   1605.0
find tag:  https://www.apartments.com/the-drake-and-anderson-court-davis-ca/8j8wf23/
2 beds   1985.0
2 beds   1985.0
2 beds   2090.0
2 beds   2220.0
2 beds   1985.0
2 beds   1985.0
2 beds   2090.0
2 beds   2220.0
find tag:  https://www.apartments.com/the-meadows-apartments-sunnyvale-ca/4fzr7tk/
1 bed   3025.0
2 beds   3650.0
1 bed   3025.0
2 beds   3650.0
Page:  25
find tag:  https://www.apartments.com/palm-view-village-torrance-ca/tt4kpfc/
2 beds   2850.0
2 beds   2950.0
2 beds   2850.0
2 beds   2950.0
find tag:  https://www.apartments.com/cedar-flats-chico-c

Studio   1913.0
1 bed   2195.0
2 beds   3201.0
Studio   1913.0
1 bed   2195.0
2 beds   3201.0
find tag:  https://www.apartments.com/via-ventura-ventura-ca/33m4jcg/
1 bed   2300.0
2 beds   2650.0
1 bed   2300.0
2 beds   2650.0
find tag:  https://www.apartments.com/the-arbors-livermore-ca/xpmb11p/
find tag:  https://www.apartments.com/bella-vista-at-hilltop-richmond-ca/m449jlj/
1 bed   1820.0
2 beds   2006.0
2 beds   2092.0
1 bed   1820.0
2 beds   2006.0
2 beds   2092.0
find tag:  https://www.apartments.com/the-hilltop-santa-cruz-ca/85pmeb0/
1 bed   2400.0
1 bed   2700.0
2 beds   3050.0
2 beds   3100.0
1 bed   2400.0
1 bed   2700.0
2 beds   3050.0
2 beds   3100.0
find tag:  https://www.apartments.com/the-highlands-at-grand-terrace-grand-terrace-ca/gewbmyy/
find tag:  https://www.apartments.com/milpa-village-townhomes-milpitas-ca/pqj7wgt/
3 beds   4095.0
3 beds   4245.0
3 beds   4245.0
3 beds   4295.0
3 beds   4295.0
3 beds   4345.0
3 beds   4345.0
3 beds   4495.0
3 beds   4095.0
3 beds  

Studio   1284.0
Studio   1294.0
Studio   1294.0
Studio   1700.0
Studio   1879.0
Studio   1284.0
Studio   1294.0
Studio   1294.0
Studio   1700.0
Studio   1879.0


In [97]:
rent_data = pd.DataFrame(all_property)
rent_data

,1 bed,2 beds,Property_Name_Rent,City_Rent,Type,State_Rent,Zip_code,Full_Address,Rent_link,Studio,3 beds,6 beds,4 beds
0,2075.0,2365.0,Solana at Grand,Escondido,Apartment,CA,92027,"1501 E Grand Ave, Escondido, CA 92027",https://www.apartments.com/solana-at-grand-esc...,NaN,NaN,NaN,NaN
1,2250.0,2680.0,The Summit at Chino Hills,Chino Hills,Apartment,CA,91709,"2400 Ridgeview Dr, Chino Hills, CA 91709",https://www.apartments.com/the-summit-at-chino...,NaN,NaN,NaN,NaN
2,2470.0,2849.0,River Terrace,Santa Clara,Apartment,CA,95054,"730-730 Agnew Rd, Santa Clara, CA 95054",https://www.apartments.com/river-terrace-santa...,NaN,NaN,NaN,NaN
3,NaN,2666.0,Park Bonita,Bonita,Apartment,CA,91902,"3511-3549 Valley Rd, Bonita, CA 91902",https://www.apartments.com/park-bonita-bonita-...,NaN,NaN,NaN,NaN
4,1609.0,2057.0,Hollypark Knolls,Inglewood,Apartment,CA,90305,"3120 Hollypark Dr, Inglewood, CA 90305",https://www.apartments.com/hollypark-knolls-in...,1655.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1975.0,2125.0,Westmont Village 55+ Community,March Air Reserve Base,Apartment,CA,92518,"16808 LeMay Dr, March Air Reserve Base, CA 92518",https://www.apartments.com/westmont-village-55...,NaN,2845.0,NaN,NaN
196,NaN,NaN,Refugio,Santa Maria,Apartment,CA,93455,"2299 Carrasco Way, Santa Maria, CA 93455",https://www.apartments.com/refugio-santa-maria...,NaN,2450.0,NaN,NaN
197,1445.0,NaN,Somerset Village,Paramount,Apartment,CA,90723,"8635 Somerset Blvd, Paramount, CA 90723",https://www.apartments.com/somerset-village-pa...,NaN,NaN,NaN,NaN
198,1175.0,1400.0,Venetian Park Apartments,Stockton,Apartment,CA,95207,"1540 Mosaic Way, Stockton, CA 95207",https://www.apartments.com/venetian-park-apart...,1075.0,NaN,NaN,NaN


In [98]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q2-2021_Apartment.com_Apartment_Data_page21-28.csv", index=False, header=True)